In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../../../data/'
group_dir_ = '../../../data/groups/low'

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

num_user = len(svd['uid'].unique())
num_user

220

In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [4]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,3,2.544071,1
1,0,7,2.545822,1
2,0,18,2.152490,1
3,0,20,1.882396,1
4,0,31,2.410806,1


In [5]:
# Count POP to DF
tid_list = []
pop_list = []
for i in df['tid'].unique():
    tid_list.append(i)
    pop_list.append(len(df[df['tid']==i])/num_user*5)

In [6]:
d = {'tid': tid_list, 'rating': pop_list}
df_pop = pd.DataFrame(data=d)
df_pop = df_pop.sort_values(by=['rating'], ascending=False)
df_pop[:10]

,tid,rating
166,166,3.840909
7209,7209,3.295455
350,350,3.113636
3761,3761,3.045455
80,80,3.022727
1017,1017,2.840909
1753,1753,2.750000
428,428,2.727273
200,200,2.681818
3159,3159,2.659091


In [7]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [14]:
for lambda_ in tqdm(lambdas):
    top_n_size = 20
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
#     print('lambda = ' + str(lambda_))
    threshold = 0.0
    avg_precisions = []
    avg_recalls = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        recalls = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                high_rating = 0
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1

                precision = high_rating / top_n_size
                recall = high_rating / len(test[test['uid'] == uid])
                precisions.append(precision)
                recalls.append(recall)
            

        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)

        avg_recall = 0
        for recall in recalls:
            avg_recall += recall
        avg_recall /= len(recalls)
        avg_recalls.append(avg_recall)
        
    print(avg_precisions)
    print(avg_recalls)
    print('----------------------------------------') 
#     print(sum(avg_precisions) / len(avg_precisions))
#     print('----------------------------------------') 

[0.08833333333333337, 0.08030303030303024, 0.06447368421052627, 0.05846153846153843, 0.06583333333333329, 0.05803571428571429, 0.05446428571428571]
[0.006198874442100916, 0.00746280880474238, 0.007720411693196416, 0.00785389544983411, 0.00864334974177376, 0.008831181151618262, 0.00873861661950641]
----------------------------------------
[0.10900000000000006, 0.08989898989898987, 0.0697368421052631, 0.06230769230769229, 0.06749999999999996, 0.05892857142857143, 0.05535714285714286]
[0.007064571349205906, 0.007991732457423855, 0.007945615660055215, 0.0079539995152103, 0.008743715519910492, 0.008852239103100741, 0.00885678585446744]
----------------------------------------
[0.13066666666666676, 0.1005050505050505, 0.07368421052631574, 0.06461538461538459, 0.07249999999999998, 0.06249999999999999, 0.05803571428571428]
[0.007548065219954845, 0.008262735183066267, 0.008287901821909, 0.008310165969135468, 0.009170491372652832, 0.009405605408505166, 0.009131298037417846]
---------------------

In [15]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    for i in range(len(r)):
        r[i] = 2**r[i] -1
    
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, r_max, k, method=0):
    dcg_max = dcg_at_k(r_max, k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [16]:
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
#     print('lambda = ' + str(lambda_))
    threshold = 0.0
    avg_nDCGs = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        nDCGs = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                truth_rating = []
                uid = group[k]
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0:
                        truth_rating.append(t.iloc[0]['rating'])
                    else:
                        truth_rating.append(0)
                max_rating = test[test['uid']==uid].sort_values(by=['rating'],  ascending=False)['rating'].values[:top_n_size]
                nDCG = ndcg_at_k(truth_rating, max_rating, top_n_size, method=1)
                nDCGs.append(nDCG)  
                
        avg_nDCG = 0
        for nDCG in nDCGs:
            avg_nDCG += nDCG
        avg_nDCG /= len(nDCGs)
        avg_nDCGs.append(avg_nDCG)
        
    print(avg_nDCGs)
#     print('----------------------------------------') 

[0.044764748758321656, 0.03565777517598908, 0.025167646137125105, 0.022372892954416097, 0.02679354193522735, 0.02121881172234608, 0.025347996467148814]
[0.05269172245107926, 0.03724120450733674, 0.026645516897922904, 0.023719136077004196, 0.026572721762676256, 0.021318934661417897, 0.025270071982814668]
[0.06280015988697102, 0.040421205450380034, 0.027885352127097706, 0.023391280363767832, 0.027894414924142984, 0.02156831823297323, 0.025450296864587045]
[0.07917929917943502, 0.044607571219197596, 0.03141466985099406, 0.025789286062992064, 0.02852729952384559, 0.021667580176446377, 0.025889813368166887]
[0.0866274264656733, 0.05010493687094735, 0.036147006943960155, 0.029054144120395153, 0.029606812267810072, 0.02305991242533729, 0.027278290589235595]
[0.09068690384326651, 0.05016566686003172, 0.03877686440783377, 0.027261852704114034, 0.028500275266553205, 0.022416261597309806, 0.026749994710141053]
[0.09107132815560781, 0.050801851862982764, 0.04177777494252275, 0.02614751010259841, 0